In [242]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
print('Libraries Imported')

Libraries Imported


In [243]:
df_iems=pd.read_csv('crinacle_iem_rankings.csv')
df_iems.sample(2)

,Rank,Value Rating,Model,Price (MSRP),Signature,Comments,Tone Grade,Technical Grade,Setup,Status,Ranksort,Tonesort,Techsort,Pricesort
459,C+,NaN,Seek Real Audio Airship,180,V-shaped,NaN,B,C,DD,Available for demo at The Hangout,168,57,111,180
1095,A,★★★,7Hz Timeless,220,Neutral with bass boost,Easily the best-tuned planar IEM by a country ...,A,A,Planar,Available for demo at The Hangout,42,21,21,220


EDA

In [244]:
print(df_iems.shape)
df_iems.info()

(2143, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2143 entries, 0 to 2142
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Rank             2143 non-null   object
 1   Value Rating     155 non-null    object
 2   Model            2143 non-null   object
 3   Price (MSRP)     2141 non-null   object
 4   Signature        2141 non-null   object
 5   Comments         1793 non-null   object
 6   Tone Grade       2141 non-null   object
 7   Technical Grade  2141 non-null   object
 8   Setup            2135 non-null   object
 9   Status           2125 non-null   object
 10  Ranksort         2143 non-null   object
 11  Tonesort         2143 non-null   object
 12  Techsort         2143 non-null   object
 13  Pricesort        2143 non-null   object
dtypes: object(14)
memory usage: 234.5+ KB


In [245]:
print(df_iems['Value Rating'].count())
df_iems.isnull().sum()

155


Rank                  0
Value Rating       1988
Model                 0
Price (MSRP)          2
Signature             2
Comments            350
Tone Grade            2
Technical Grade       2
Setup                 8
Status               18
Ranksort              0
Tonesort              0
Techsort              0
Pricesort             0
dtype: int64

Drop the "Value Rating" as most of it is null

In [246]:
df_iems=df_iems.drop(columns='Value Rating')
df_iems.sample(2)

,Rank,Model,Price (MSRP),Signature,Comments,Tone Grade,Technical Grade,Setup,Status,Ranksort,Tonesort,Techsort,Pricesort
1929,C-,Audiosense AQ7,500,Unique,"Sub-bass boost, odd midrange and rolled-off tr...",D+,C+,6BA 1DD,Available for demo at The Hangout,248,157,91,500
1141,B+,HYLA TE-5B,1300,U-shaped,Similar to the CE-5 though ever-so-slightly th...,B+,A,1Piezo 2BA 1DD,Zeppelin & Co demo unit,64,43,21,1300


Drop all unecessary hidden column from the original data

In [247]:
df_iems=df_iems.drop(columns=['Ranksort','Tonesort','Techsort','Pricesort'])
df_iems.sample(2)

,Rank,Model,Price (MSRP),Signature,Comments,Tone Grade,Technical Grade,Setup,Status
1742,C,Meze 12 Classics,70,V-shaped,Generic V-shape tuning with middling technical...,C,C,DD,AV One demo unit
131,B,Fearless Lancelot,1300,Warm,Rather generic tuning with no redeeming qualit...,B,A-,2EST 6BA,Music Sanctuary demo unit


Exploring Price Column

In [248]:
#Renaming it for better calling

df_iems.rename({'Price (MSRP)':'Price'}, axis=1, inplace=True)
df_iems.sample(2)

,Rank,Model,Price,Signature,Comments,Tone Grade,Technical Grade,Setup,Status
1111,A-,Kumitate KL-Sirius,815,Neutral,"Excellent technical ability, well tuned with a...",A,A-,4BA,Third party custom
39,A-,Kumitate KL-Sirius,815,Neutral,"Excellent technical ability, well tuned with a...",A,A-,4BA,Third party custom


In [249]:
#Convert the price column to numeric, non number are marked as null
df_iems['Price']=pd.to_numeric(df_iems['Price'], errors='coerce') #stored it in float
df_iems.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2143 entries, 0 to 2142
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Rank             2143 non-null   object 
 1   Model            2143 non-null   object 
 2   Price            1886 non-null   float64
 3   Signature        2141 non-null   object 
 4   Comments         1793 non-null   object 
 5   Tone Grade       2141 non-null   object 
 6   Technical Grade  2141 non-null   object 
 7   Setup            2135 non-null   object 
 8   Status           2125 non-null   object 
dtypes: float64(1), object(8)
memory usage: 150.8+ KB


In [250]:
df_iems['Price'].isnull().sum()
df_iems.dropna(subset=['Price'], inplace=True)
display(df_iems.shape)
df_iems.isna().sum()


(1886, 9)

Rank                 0
Model                0
Price                0
Signature            0
Comments           336
Tone Grade           0
Technical Grade      0
Setup                4
Status              12
dtype: int64

In [251]:
warnings.filterwarnings('ignore')
df_iems['Comments'].fillna('No Comments', inplace=True)
df_iems['Status'].fillna('No Status', inplace=True)
df_iems.isna().sum()

Rank               0
Model              0
Price              0
Signature          0
Comments           0
Tone Grade         0
Technical Grade    0
Setup              4
Status             0
dtype: int64

Removing Duplicate rows according to models and price

In [252]:
duplicated_rows=df_iems[df_iems.duplicated(subset=['Model', 'Price'], keep=False)].sort_values(by=['Model','Price'])
print(df_iems.duplicated(subset=['Model', 'Price']).sum())
display(duplicated_rows)

943


,Rank,Model,Price,Signature,Comments,Tone Grade,Technical Grade,Setup,Status
485,C+,1Custom SA03,300.0,Warm V-shape,Warm lower-mid response balanced out by a forw...,C+,C+,3BA,Jaben SG demo unit
1557,C+,1Custom SA03,300.0,Warm V-shape,Warm lower-mid response balanced out by a forw...,C+,C+,3BA,Jaben SG demo unit
974,D,1More Piston,15.0,V-shaped,"Tonally alright, technically distrastrous.",C-,D,DD,AV One demo unit
2046,D,1More Piston,15.0,V-shaped,"Tonally alright, technically distrastrous.",C-,D,DD,AV One demo unit
841,C-,1More Quad Driver,200.0,"""Balanced""",Okay-ish technical ability in a mainstream tun...,C,C-,3BA 1DD,Jaben Melbourne demo unit
...,...,...,...,...,...,...,...,...,...
1335,B-,qdc Neptune,200.0,Warm neutral,Tuning and configuration is simple but goes a ...,B,B-,BA,Music Sanctuary demo unit
71,B+,qdc Tiger,1500.0,Mild V-shape,No Comments,B+,A,2EST 6BA,e-Earphone demo unit
1143,B+,qdc Tiger,1500.0,Mild V-shape,No Comments,B+,A,2EST 6BA,e-Earphone demo unit
393,C+,qdc Uranus,200.0,Warm,Sounds like another variation of the qdc Fusio...,C+,B-,1BA 1DD,Music Sanctuary demo unit


In [253]:
df_iems = df_iems.drop_duplicates(subset=['Model', 'Price'], keep='first')
print(df_iems.duplicated(subset=['Model', 'Price']).sum())
df_iems.shape
df_iems[df_iems['Model']=='1More Piston']

0


,Rank,Model,Price,Signature,Comments,Tone Grade,Technical Grade,Setup,Status
974,D,1More Piston,15.0,V-shaped,"Tonally alright, technically distrastrous.",C-,D,DD,AV One demo unit


Treating Setup column

In [254]:
setup_list=df_iems[df_iems['Setup'].isna()].index.tolist()
print(setup_list)
print(type(setup_list))
df_iems[df_iems['Setup'].isna()]

[80, 1033]
<class 'list'>


,Rank,Model,Price,Signature,Comments,Tone Grade,Technical Grade,Setup,Status
80,B+,Fatfreq Maestro SE,1900.0,Neutral with bass boost,No Comments,A,B+,NaN,Showfloor demo unit
1033,D-,Rhapsodio Clipper,55.0,Bassy,Lots and lots of bass. Slams you in the face w...,D,D-,NaN,Zeppelin & Co demo unit


In [255]:
setup_input_list = ['1DD 7BA 4EST', '1DD']

df_iems.loc[setup_list, 'Setup'] = setup_input_list
display(df_iems.loc[setup_list])
    

,Rank,Model,Price,Signature,Comments,Tone Grade,Technical Grade,Setup,Status
80,B+,Fatfreq Maestro SE,1900.0,Neutral with bass boost,No Comments,A,B+,1DD 7BA 4EST,Showfloor demo unit
1033,D-,Rhapsodio Clipper,55.0,Bassy,Lots and lots of bass. Slams you in the face w...,D,D-,1DD,Zeppelin & Co demo unit


In [256]:
df_iems.isna().sum()

Rank               0
Model              0
Price              0
Signature          0
Comments           0
Tone Grade         0
Technical Grade    0
Setup              0
Status             0
dtype: int64

All Null Values and duplicated rows Treated here

Encoding Grades


In [257]:
df_iems['Rank'].unique()

array(['S-', 'A+', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D+', 'D',
       'D-', 'E', 'F'], dtype=object)

In [258]:
grade_map={
    'S+':13, 'S':12, 'S-':11,
    'A+':10, 'A':9, 'A-':8,
    'B+':7, 'B':6, 'B-':5,
    'C+':4, 'C':3, 'C-':2,
    'D+':1, 'D':0, 'D-':-1,
    'E+':-2, 'E':-3, 'E-':-4,
    'F+':-5, 'F':-6, 'F-':-7
}

In [259]:
#mapping with the dict made above
df_iems['Rank_encoded']=df_iems['Rank'].map(grade_map)
df_iems['Technical_Grade_encoded']=df_iems['Technical Grade'].map(grade_map)
df_iems['Tone_Grade_encoded']=df_iems['Tone Grade'].map(grade_map)
#here to display the result to check if it is encoded
display(df_iems[['Rank', 'Rank_encoded', 'Tone Grade', 'Tone_Grade_encoded', 'Technical Grade', 'Technical_Grade_encoded']].head())

,Rank,Rank_encoded,Tone Grade,Tone_Grade_encoded,Technical Grade,Technical_Grade_encoded
0,S-,11,S-,11,S,12
1,S-,11,S+,13,A+,10
2,S-,11,S+,13,A+,10
3,A+,10,A+,10,S,12
4,A+,10,A+,10,S-,11


In [260]:
df_iems.sample(2)

,Rank,Model,Price,Signature,Comments,Tone Grade,Technical Grade,Setup,Status,Rank_encoded,Technical_Grade_encoded,Tone_Grade_encoded
704,C,Audio Technica ATH-ANC300TW,200.0,V-shaped,"Weirdly bloated, weirdly sibilant, not the bes...",C,C,DD,Zeppelin & Co demo unit,3,3,3
578,C,Custom Art FIBAE 2,550.0,Warm,Slight downsloping response with a generally p...,C,C+,2BA,Music Sanctuary demo unit,3,4,3


Trying Ordinal Encoder form sklearn

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
import pandas as pd

# 1. Define the correct order for your grades, from lowest to highest
#    This must be a list of lists, one for each column you're transforming.
grade_order=['D-', 'D', 'D+', 'C-', 'C', 'C+', 'B-', 'B', 'B+', 'A-', 'A', 'A+', 'S-', 'S', 'S+']

# 2. Create the encoder, passing the correct order
#    Let's assume 'Rank' and 'Technical Grade' use this order.
encoder=OrdinalEncoder(categories=[grade_order, grade_order])

# 3. Apply the encoder to the desired columns
#    Note: The encoder requires a 2D array, so we use double brackets [[]].
encoded_columns=encoder.fit_transform(df_iems[['Rank', 'Technical Grade', 'Tone Grade']])

# 4. Add the new encoded columns back to the DataFrame
df_iems[['Rank_encoded', 'Technical_Grade_encoded']]=encoded_columns

print(df_iems[['Rank', 'Rank_encoded', 'Technical Grade', 'Technical_Grade_encoded']].head())